In [1]:
from pyomo.environ import *
import numpy as np

# Definir el modelo
model = ConcreteModel()

# Conjunto de generadores
N = 3  # Número de generadores
model.G = RangeSet(1, N)

# Parámetros
Pd = 850  # Demanda total en MW
B = np.array([[0.00007, 0.00010, 0.00012],  # Coeficientes B_ij de pérdidas
              [0.00010, 0.00008, 0.00011],
              [0.00012, 0.00011, 0.00009]])

B0 = np.array([0.0003, 0.00025, 0.0002])  # Coeficientes B_i0
B00 = 0.0005  # Coeficiente B_00

# Coeficientes de costo
a = {1: 0.002, 2: 0.0018, 3: 0.0025}
b = {1: 10, 2: 8, 3: 6}
c = {1: 100, 2: 120, 3: 150}

# Parámetros del efecto de punto de válvula
d = {1: 20, 2: 18, 3: 25}
e = {1: 0.03, 2: 0.025, 3: 0.02}

# Límites de generación
Pmin = {1: 150, 2: 100, 3: 50}
Pmax = {1: 500, 2: 400, 3: 200}

# Límites de rampa
UR = {1: 50, 2: 40, 3: 30}
DR = {1: 50, 2: 40, 3: 30}

# Variables de potencia de generación
model.PG = Var(model.G, within=NonNegativeReals)

# Función de costo total
def cost_function(model):
    return sum(a[i] * model.PG[i]**2 + b[i] * model.PG[i] + c[i] + 
               abs(d[i] * sin(e[i] * (Pmin[i] - model.PG[i]))) for i in model.G)
model.cost = Objective(rule=cost_function, sense=minimize)

# Restricción de balance de potencia
def power_balance_rule(model):
    Ploss = sum(sum(model.PG[i] * B[i-1, j-1] * model.PG[j] for j in model.G) for i in model.G) + \
            sum(B0[i-1] * model.PG[i] for i in model.G) + B00
    return sum(model.PG[i] for i in model.G) == Pd + Ploss
model.power_balance = Constraint(rule=power_balance_rule)

# Restricciones de generación
def generation_limits_rule(model, i):
    return (Pmin[i], model.PG[i], Pmax[i])
model.gen_limits = Constraint(model.G, rule=generation_limits_rule)

# Restricciones de rampa
def ramp_up_rule(model, i):
    return model.PG[i] - Pmin[i] >= -DR[i]
model.ramp_up = Constraint(model.G, rule=ramp_up_rule)

def ramp_down_rule(model, i):
    return Pmax[i] - model.PG[i] >= UR[i]
model.ramp_down = Constraint(model.G, rule=ramp_down_rule)

# Resolver el modelo
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)

# Mostrar resultados
print("\nResultados de generación óptima:")
for i in model.G:
    print(f"Generador {i}: {model.PG[i]():.2f} MW")

print(f"\nCosto total: {model.cost():.2f} $/h")


ipopt


ApplicationError: No executable found for solver 'ipopt'